In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

rng = np.random.default_rng(42)

n = 120
p_demand = 0.12
sizes = rng.gamma(shape=2.0, scale=8.0, size=n)
occurs = rng.random(n) < p_demand

y = np.where(occurs, sizes, 0.0)
index = pd.period_range("2015-01", periods=n, freq="M")
series = pd.Series(y, index=index, name="Demand")

fig = go.Figure()
fig.add_trace(go.Scatter(x=series.index.to_timestamp(), y=series, mode="lines+markers", name="Demand"))
fig.update_layout(title="Intermittent Demand Example", xaxis_title="Time", yaxis_title="Units")
fig.show()


In [2]:
def croston_forecast(y, alpha=0.1):
    y = np.asarray(y, dtype=float)
    n = len(y)
    nz = np.where(y > 0)[0]
    if len(nz) == 0:
        return np.zeros(n), np.zeros(n), np.zeros(n)

    first = nz[0]
    z = y[first]
    p = 1.0
    last = first

    z_hist = np.zeros(n)
    p_hist = np.zeros(n)
    f_hist = np.zeros(n)

    for t in range(first, n):
        if y[t] > 0:
            interval = max(1, t - last)
            z = alpha * y[t] + (1 - alpha) * z
            p = alpha * interval + (1 - alpha) * p
            last = t
        z_hist[t] = z
        p_hist[t] = p
        f_hist[t] = z / p

    return f_hist, z_hist, p_hist

f_croston, z_hist, p_hist = croston_forecast(series.values, alpha=0.2)


In [3]:
def tsb_forecast(y, alpha=0.1, beta=0.1):
    y = np.asarray(y, dtype=float)
    n = len(y)
    nz = np.where(y > 0)[0]
    if len(nz) == 0:
        return np.zeros(n), np.zeros(n), np.zeros(n)

    first = nz[0]
    z = y[first]
    p = 1.0  # probability of demand

    z_hist = np.zeros(n)
    p_hist = np.zeros(n)
    f_hist = np.zeros(n)

    for t in range(first, n):
        demand = 1.0 if y[t] > 0 else 0.0
        if demand > 0:
            z = alpha * y[t] + (1 - alpha) * z
        p = beta * demand + (1 - beta) * p

        z_hist[t] = z
        p_hist[t] = p
        f_hist[t] = p * z

    return f_hist, z_hist, p_hist

f_sba = f_croston * (1 - 0.2 / 2)
f_tsb, z_tsb, p_tsb = tsb_forecast(series.values, alpha=0.2, beta=0.2)

fig = go.Figure()
fig.add_trace(go.Scatter(x=series.index.to_timestamp(), y=series, mode="lines", name="Demand"))
fig.add_trace(go.Scatter(x=series.index.to_timestamp(), y=f_croston, mode="lines", name="Croston"))
fig.add_trace(go.Scatter(x=series.index.to_timestamp(), y=f_sba, mode="lines", name="SBA"))
fig.add_trace(go.Scatter(x=series.index.to_timestamp(), y=f_tsb, mode="lines", name="TSB"))
fig.update_layout(title="Intermittent Demand Forecasts", xaxis_title="Time", yaxis_title="Units")
fig.show()


In [4]:
try:
    from sktime.forecasting.croston import Croston, SBA, TSB
    from sktime.datasets import load_airline

    y = load_airline()
    fh = [1, 2, 3, 4, 5, 6]

    forecaster = Croston(smoothing=0.2)
    forecaster.fit(y)
    y_pred = forecaster.predict(fh)
    y_pred.head()
except Exception as e:
    print("sktime optional demo skipped:", e)


sktime optional demo skipped: No module named 'sktime'
